# CogPonder: N-Back Task with Fixed Hyper-parameters

This notebook trains a single-task CogPonder agent on N-back task. The operator is a spatiotemporal encoder (CNN+LSTM) with fixed hyper-parameters and a binary classification head.

## Data

Here, we only use the 2-back data from the *Self-Regulation Ontology* dataset ([see `data/Self_Regulation_Ontology/`](../data/Self_Regulation_Ontology/README.md)).

#### 2-back

The input includes recent N+1 symbols, including the one presented in the current trial; the last input is the current symbol. The output is the response to the current trial (either match or non-match).


In [1]:
%reload_ext autoreload
%autoreload 3

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

import torch
import pytorch_lightning as pl
import pandas as pd
from pytorch_lightning.callbacks import EarlyStopping
from src.cogponder import CogPonderModel
from src.cogponder.data import NBackSRODataset, CogPonderDataModule
from pathlib import Path

In [2]:
# Parameters

# number of maximum epochs to train (early stopping will be applied)
# early stopping patience is 10% of max_epochs (min 10 epochs)
max_epochs = 1000
batch_size = 1024
step_duration = 20

In [3]:
# Data

print(f'Loading N-back dataset... ', end='')

dataset = NBackSRODataset(n_subjects=1, n_back=2, step_duration=step_duration)
datamodule = CogPonderDataModule(dataset, batch_size=batch_size, num_workers=8)
datamodule.prepare_data()
# determine some parameters from data
n_subjects = torch.unique(dataset[:][0]).size(0)
n_contexts = torch.unique(dataset[:][2]).size(0)
n_features = dataset[:][3].size(-1)
n_outputs = torch.unique(dataset[:][4]).size(0)
max_response_step = dataset[:][5].max().int().item() + 1

configs = {
    'inputs_dim': n_features,
    'outputs_dim': n_outputs,
    'embeddings_dim': 8,
    'time_loss_beta': 10.,
    'learning_rate': 1e-2,
    'max_response_step': max_response_step,
    'n_contexts': n_contexts,
    'n_subjects': n_subjects,
    'subject_embeddings_dim': 2,
    'task': 'nback',
    'operator_type': 'spatiotemporal'
}

print('Done!')

Loading N-back dataset... Done!


In [4]:
# Experiment

model = CogPonderModel(**configs)

# model = torch.compile(model)

# Trainer
trainer = pl.Trainer(
    max_epochs=max_epochs,
    min_epochs=100,
    accelerator='cpu',
    log_every_n_steps=1,
    # overfit_batches=True,
    # accumulate_grad_batches=2,
    callbacks=[
        EarlyStopping(monitor='val/total_loss',
                      patience=100,
                      mode='min', min_delta=0.001),
    ])

# Fit and evaluate the model
trainer.fit(model, datamodule=datamodule)

# save checkpoint
checkpoint_path = f'models/checkpoints/nback/cogponder_epochs-{trainer.current_epoch}.ckpt'
trainer.save_checkpoint(checkpoint_path)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type                         | Params
---------------------------------------------------------------
0 | halt_node     | HaltingModule                | 81    
1 | iterator_node | IteratorModule               | 314   
2 | operator_node | SpatioTemporalOperatorModule | 371   
3 | embeddings    | Embedding                    | 16    
4 | resp_loss_fn  | ResponseLoss                 | 0     
5 | time_loss_fn  | ResponseTimeLoss             | 0     
---------------------------------------------------------------
782       Trainable params
0         Non-trainable params
782       Total params
0.003     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/morteza/micromamba/envs/cogponder/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
# DEBUG

model_ckpt = CogPonderModel.load_from_checkpoint(checkpoint_path)